# SUMÁRIO DO PROJETO




1. Qual o contexto da problemática? O que queremos resolver?
  - Empresa de Streamming de Música: Ela tem uma abordagem reativa em relação a clientes que não renovam a assinatura oferecendo 3 meses grátis para os que não renovam. Foi entendido que uma abordagem proativa pode ser mais efetiva.

2. Quais DADOS temos para agregar na solução?
  - Temos um dicionario de dados a seguir.

3. Quais INFORMAÇÕES podemos extrair desses DADOS?
  - aqui irei analisar as possiveis variaveis que podem ser criadas (Feature Engineering)

4. Como chegar na solução e quais métodos/tecnologias podem apoiar e quais não podem? (justificação dos usos, questões de negócio, performance, processamento computacional, recursos e implantação em uma hipotética produtização)
  - aqui escrevo sobre ....

5. Ótica de negócios: Ganhos $$
  - a


para encaixar no sumário:

1- EDA
- hipoteses voltadas ao problema e para a modelagem
- implicações das análises
- usar o EDA para tomada de decisões

2- Dataprep + feature engeneering + feature selection
- criação do publico-alvo, target, análise de volumetrias, missing, outliers, etc.
- construção de novas features / transformação de features
- feature selection adequado, variaveis significativas para o problema

3- Modelagem
- Estrutura do modelo utilizado, funcionamento, hiperparametros, vantagens e fraquezas
- Modelagem e pipe line adequados ao problema? eu entendo isso?
- Compreendo os impactos ao utilizar outras tecnicas de modelagem e pipeline construidos?

4- Validação
- Validação adequada? (Leakage, overfit, underfit, tamanho da amostra)
- Formas de validação exploradas? (cross-validation, train test split, out of sample, out of time, etc)
- Como as alterações do problema impactam na validação escolhida?

5- Definição e avaliação dos resultados
- Métricas de avaliação adequadas? (MAE, RMSE, Recall, AUC, Silhueta, etc)
- Como outras métricas impactariam os resultados do case?
- Como mudanças na estrutura do problema impactariam diferentes métricas?

6- Qualidade do código
- Organização e boas praticas do codigo (nomes intuitivos, comentários, ordenamento intuitivo)
- Codigo padronizado? (classes, funções e variaveis com nomes padronizados, clean code, pep8, etc?)
- Código visa a implementação? (modularização, tempo de processamento, versionamento etc?)

7- Soft Skills
- Sucinto e dentro do tempo indicado
- Clareza nas explicações
- Apresentação estruturada (começo, meio e fim)


# Instalação do ambiente inicial

Conectando com o Google Drive (armazenamento)

Instalação do Java

Instalação do Apache Spark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark

# Configurando o Spark

Configuração das variaveis ambiente

In [ ]:
# configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"

# tornar o pyspark "importável"
import findspark
findspark.init()

# iniciar uma sessão local
from pyspark.sql import SparkSession
# spark = SparkSession.builder.master('local[*]').appName('Case_dataMaster_Victor').getOrCreate()

spark = SparkSession.builder \
                    .appName("Case_dataMaster_Victor") \
                    .config("spark.executor.memory", "4g") \
                    .config("spark.driver.memory", "4g") \
                    .config("spark.executor.cores", "2") \
                    .config("spark.driver.maxResultSize", "2g") \
                    .getOrCreate()

     #  # Memória do executor
     # Memória do driver
     # Número de núcleos
     # Tamanho máximo do resultado

# Importando os dados da RAW

Dados baixados e localizados no google drive para nao tem repetição em acessar o ambiente do kaggle e baixar os dados de la

In [ ]:
path_members = '/content/drive/MyDrive/Colab Notebooks/data_master_VICTOR/raw/members.parquet'
path_transactions = '/content/drive/MyDrive/Colab Notebooks/data_master_VICTOR/raw/transactions.parquet'
path_userlogs = '/content/drive/MyDrive/Colab Notebooks/data_master_VICTOR/raw/user_logs.parquet'

df_members = (spark.read
                   .format("parquet")
                   .option("header", True)
                   .option("inferSchema", True)
                   .load(path_members))

df_transactions = (spark.read
                   .format("parquet")
                   .option("header", True)
                   .option("inferSchema", True)
                   .load(path_transactions))

df_userlogs = (spark.read
                   .format("parquet")
                   .option("header", True)
                   .option("inferSchema", True)
                   .load(path_userlogs))

## Trazendo para uma view temporaria para consultas SQL

In [ ]:
df_members.createOrReplaceTempView('tb_members')
df_transactions.createOrReplaceTempView('tb_transactions')
df_userlogs.createOrReplaceTempView('tb_userlogs')

# Inicio de EDA (Exploratory Data and Analysis)



## Dicionario de dados INICIAL

### 1. Transactions:
  - Colunas:
    - 1º 'mnso': id do usuário;
    - 2º 'payment_method_id': Método de pagamento;
    - 3º 'payment_plan_days': tamanho do plano em DIAS;
    - 4º 'plan_list_price': preço do respectivo plano em New Taiwan Dollar (NTD);
    - 5º 'actual_amount_paid': Total pago atual (nao entendi);
    - 6º 'is_auto_renew': Renovação automatica 1 ou 0;
    - 7º 'transaction_date': data da transação no formato %Y %M %D;
    - 8º 'membership_expire_date': quando o plano vence no formato %Y %M %D;
    - 9º 'is_cancel': se nessa transação o cliente cancelou ou não o plano;
    - 10º 'safra': data (Data foto da base).

### 2. User_logs:
  - Colunas:
    - 1º 'msno': id do usuário;
    - 2º 'safra': formato %Y %M %D (data foto da base);
    - 3º 'num_25': quantidade de músicas tocadas menos de 25% da duração dela;
    - 4º 'num_50': quantidade de músicas tocadas entre 25% e 50% da duração dela;
    - 5º 'num_75': quantidade de músicas tocadas entre 50% e 75% da duração dela;
    - 6º 'num_985': quantidade de músicas tocadas entre 75% e 98,5% da duração dela;
    - 7º 'num_100': quantidade de músicas tocadas acima de 98,5%;
    - 8º 'num_unq': quantidade de musicas tocadas apenas 1 vez;
    - 9º 'total_secs': total de segundos tocados.

### 3. Members:
  - Colunas:
    - 1º 'msno': id do usuário;
    - 2º 'city': cidade do usuário?
    - 3º 'bd': idade? obs: outliers em 2015
    - 4º 'gender': gênero
    - 5º 'registered_via': meio de registro do cliente
    - 6º 'registration_init_time': primeiro regristo do cliente formato %Y %M %D
    - 7º 'safra': Data da foto da base
    - 8º 'is_ativo': se o cliente esta ativo ou não

## Importando bibliotecas e funções necessárias para a exploração

In [ ]:
from pyspark.sql.functions import col, countDistinct, count, min, max, when, to_date, year

from pyspark.sql.types import *
from pyspark.sql.functions import *

# import select functions and types
from pyspark.sql.types import IntegerType, StringType
from pyspark.sql.functions import floor, round

## Entendendo o formato dos dados

In [ ]:
table = ['tb_members', 'tb_transactions', 'tb_userlogs']

for i in table:
  print(f'--------Tabela: {i}-------------------------')
  qnt_linhas = spark.sql(f'SELECT count(*) AS quantidade_linhas from {i}').collect()[0]['quantidade_linhas']
  df = spark.sql(f"SELECT * from {i}")
  qnt_colunas = len(df.columns)
  print(f"Formato da {i}: ", qnt_colunas, " X ", qnt_linhas )
  spark.sql(f'Describe {i}').show()

--------Tabela: tb_members-------------------------
Formato da tb_members:  8  X  63867246
+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                msno|   string|   null|
|               safra|   string|   null|
|registration_init...|   string|   null|
|                city|   string|   null|
|                  bd|   string|   null|
|              gender|   string|   null|
|      registered_via|   string|   null|
|            is_ativo|      int|   null|
+--------------------+---------+-------+

--------Tabela: tb_transactions-------------------------
Formato da tb_transactions:  10  X  20712225
+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                msno|   string|   null|
|   payment_method_id|   string|   null|
|   payment_plan_days|   string|   null|
|     plan_list_price|   string|   null|
|  actual_amount_paid|   st

## 1 - Entendendo a Members

### Primeira visão

In [ ]:
spark.sql('select * from tb_members LIMIT 10').show()

+--------------------+------+----------------------+----+---+------+--------------+--------+
|                msno| safra|registration_init_time|city| bd|gender|registered_via|is_ativo|
+--------------------+------+----------------------+----+---+------+--------------+--------+
|+++snpr7pmobhLKUg...|201612|            2014-09-27|   1|  0|  null|             7|    true|
|++/AwGzubug3gT6J+...|201607|            2015-03-22|   1|  0|  null|             9|   false|
|++/Gw1B9K+XOlB3hL...|201601|            2012-12-17|  15| 32|  male|             3|    true|
|++02XbtviomSxcIBU...|201603|            2013-11-12|  14| 21|  male|             7|   false|
|++0O0Bq04sB/9ZcOS...|201610|            2014-10-21|   5| 33|  male|             3|   false|
|++2AQgVgYUAqJDw68...|201608|            2015-04-16|  18| 23|  male|             3|   false|
|++2gRJ7i2MbO6qUG6...|201608|            2014-06-16|  11| 33|  male|             7|   false|
|++3brN43Yd6GURegT...|201604|            2010-05-12|  14| 19|  male|  

Nas primeiras 10 linhas ja vemos a idade = 0, e valores nulos no genero

### Min e Max de datas das colunas de datas para ver até onde vão

convertendo os tipos de dados

In [ ]:
df_members = (df_members
              .withColumn('safra', col('safra').cast('int'))
              .withColumn('registration_init_time', to_date(col('registration_init_time'), 'yyyyMMdd'))
              .withColumn('bd', col('bd').cast('int'))
              .withColumn('registered_via', col('registered_via').cast('int'))
              .withColumn('is_ativo', col('is_ativo').cast('boolean'))
              )
df_members.createOrReplaceTempView('tb_members')

(spark.sql(
    """
    select min(registration_init_time), max(registration_init_time),
           min(safra), max(safra)
    from tb_members
    """
  ).show()
)

+---------------------------+---------------------------+----------+----------+
|min(registration_init_time)|max(registration_init_time)|min(safra)|max(safra)|
+---------------------------+---------------------------+----------+----------+
|                 2004-03-26|                 2016-12-31|    201601|    201612|
+---------------------------+---------------------------+----------+----------+



Temos clientes que foram registrados 2016-12-31, e clientes antigos 2004.

Temos uma base de usuarios de referente ao ano (safra) de 2016. fotos dos clientes foram tiradas a cada mês, total de 12 fotos (possiveis features stores)

### Entendendo a quantidade de usuarios por safra e repetição de Clientes

In [ ]:
(spark.sql(
    """
    select safra,
            count(msno) AS quantidade
    from tb_members
    group by 1
    order by 1
    """
    ).show()
)

+------+----------+
| safra|quantidade|
+------+----------+
|201601|   4294184|
|201602|   4502354|
|201603|   4697972|
|201604|   4876301|
|201605|   5060705|
|201606|   5250064|
|201607|   5439337|
|201608|   5615921|
|201609|   5778986|
|201610|   5949288|
|201611|   6114345|
|201612|   6287789|
+------+----------+



vendo a quantidade de ids por safra, se for igual a quantidade total, nao se repetem dentro de cada safra

In [ ]:
(spark.sql(
    """
    select distinct count(msno) AS primeria_safra
    from tb_members
    where safra = '201601'
    """
    ).show()
)

+--------------+
|primeria_safra|
+--------------+
|       4294184|
+--------------+



In [ ]:
(spark.sql(
    """
    select distinct count(msno) AS ultima_safra
    from tb_members
    where safra = '201612'
    """
    ).show()
)

+------------+
|ultima_safra|
+------------+
|     6287789|
+------------+



A cada safra não há clientes duplicados, só se repetem por safra, ou seja, no máximo 12 vezes

In [ ]:
(spark.sql(
    """
    WITH contagem_msno as
    (
      select msno,
             count(msno) AS quantidade
      from tb_members
      group by 1
    )
    select min(quantidade) as minimo,
           max(quantidade) as maximo
    from contagem_msno
    """
    ).show()
)

+------+------+
|minimo|maximo|
+------+------+
|     1|    12|
+------+------+



confirmado. Max de repetição 12 e minimo 1

### Entendendo a quantidade de ativos e não ativos por safra

In [ ]:
(spark.sql(
    """
    WITH contagem_total as (
      select safra,
             count(msno) AS quantidade
      from tb_members
      group by 1
      order by 1
    ),
      contagem_ativos as(
      select safra,
             count(is_ativo) AS quantidade
      from tb_members
      where is_ativo = 1
      group by 1
      order by 1
    ),
      contagem_nao_ativos as(
      select safra,
             count(is_ativo) AS quantidade
      from tb_members
      where is_ativo = 0
      group by 1
      order by 1
    ),
    join as (
      select contagem_total.safra,
            contagem_ativos.quantidade AS quantidade_ativos,
            contagem_nao_ativos.quantidade AS quantidade_nao_ativos,
            contagem_total.quantidade AS quantidade_total
      from contagem_total
      join contagem_ativos
      on contagem_total.safra = contagem_ativos.safra
      join contagem_nao_ativos
      on contagem_ativos.safra = contagem_nao_ativos.safra
      order by 1
    )
    select (*),
           CONCAT(ROUND((quantidade_ativos / quantidade_total) * 100, 1), '%') as taxa_ativos,
           CONCAT(ROUND((quantidade_nao_ativos / quantidade_total) * 100, 1), '%') as taxa_nao_ativos
    from join
    """
    ).show()
)

+------+-----------------+---------------------+----------------+-----------+---------------+
| safra|quantidade_ativos|quantidade_nao_ativos|quantidade_total|taxa_ativos|taxa_nao_ativos|
+------+-----------------+---------------------+----------------+-----------+---------------+
|201601|           888663|              3405521|         4294184|      20.7%|          79.3%|
|201602|           918116|              3584238|         4502354|      20.4%|          79.6%|
|201603|           871651|              3826321|         4697972|      18.6%|          81.4%|
|201604|           831565|              4044736|         4876301|      17.1%|          82.9%|
|201605|           849653|              4211052|         5060705|      16.8%|          83.2%|
|201606|           844884|              4405180|         5250064|      16.1%|          83.9%|
|201607|           967936|              4471401|         5439337|      17.8%|          82.2%|
|201608|           980643|              4635278|         561

maior parte dos clientes por safra são não ativos, indicando um possivel desbalanceamento de classes futuras.

### Entendendo os valores de cada coluna da Members

In [ ]:
def executa_query_member_por_coluna(coluna):
  query = f"""
            select count(distinct {coluna}) as quantidade_distinta_{coluna}
            from tb_members
           """
  spark.sql(query).show()

for i in df_members.columns:
    print(i)
    executa_query_member_por_coluna(i)

msno
+------------------------+
|quantidade_distinta_msno|
+------------------------+
|                 6287789|
+------------------------+

safra
+-------------------------+
|quantidade_distinta_safra|
+-------------------------+
|                       12|
+-------------------------+

registration_init_time
+------------------------------------------+
|quantidade_distinta_registration_init_time|
+------------------------------------------+
|                                      4663|
+------------------------------------------+

city
+------------------------+
|quantidade_distinta_city|
+------------------------+
|                      21|
+------------------------+

bd
+----------------------+
|quantidade_distinta_bd|
+----------------------+
|                   385|
+----------------------+

gender
+--------------------------+
|quantidade_distinta_gender|
+--------------------------+
|                         2|
+--------------------------+

registered_via
+------------------------

### Vendo se há valores missing

In [ ]:
# repetindo o codigo a cima em sql agora

(spark.sql(
    """
      SELECT
        SUM(CASE WHEN msno IS NULL THEN 1 ELSE 0 END) AS msno,
        SUM(CASE WHEN safra IS NULL THEN 1 ELSE 0 END) AS safra,
        SUM(CASE WHEN registration_init_time IS NULL THEN 1 ELSE 0 END) AS registration_init_time,
        SUM(CASE WHEN city IS NULL THEN 1 ELSE 0 END) AS city,
        SUM(CASE WHEN bd IS NULL THEN 1 ELSE 0 END) AS bd,
        SUM(CASE WHEN gender IS NULL THEN 1 ELSE 0 END) AS gender,
        SUM(CASE WHEN registered_via IS NULL THEN 1 ELSE 0 END) AS registered_via,
        SUM(CASE WHEN is_ativo IS NULL THEN 1 ELSE 0 END) AS is_ativo
      FROM tb_members
    """
  ).show()
)

+----+-----+----------------------+----+---+--------+--------------+--------+
|msno|safra|registration_init_time|city| bd|  gender|registered_via|is_ativo|
+----+-----+----------------------+----+---+--------+--------------+--------+
|   0|    0|                     0|   0|  0|38210177|             0|       0|
+----+-----+----------------------+----+---+--------+--------------+--------+



38 milhoes de registros vazios da coluna de genero, de um total de 63 milhoes

###  Observações e Questionamentos

 - Clientes cadastrados a mais de 10 anos (4663 dias distintos)
 - 21 cidades, avaliar se tem correlação com Churn
 - 385 idades distintas? algo esta muito errado.
 - 2 generos e mais da metade missing (analisar como essa variavel pode se comportar para o modelo)
 - 17 tipos de jeitos de se registrar diferentes



### Avaliando as idades 'bd'

In [ ]:
(spark.sql(
    """
      select min(bd) as minimo,
             max(bd) as maximo
      from tb_members
    """
  ).show()
)

+------+------+
|minimo|maximo|
+------+------+
| -7168|  2016|
+------+------+



#### Entendendo brevemente a distribuição dos dados

In [ ]:
(spark.sql(
    """
    SELECT
        safra,
        min(bd) AS minimo_per_safra,
        max(bd) AS maximo_per_safra,
        SUM(CASE WHEN bd IS NULL THEN 0 ELSE 1 END) AS bd_total,
        SUM(CASE WHEN bd <= 0 THEN 1 ELSE 0 END) AS bd_menor_que_0,
        SUM(CASE WHEN bd > 120 THEN 1 ELSE 0 END) AS bd_maior_que_120,
        SUM(CASE WHEN bd > 0 AND bd <= 120 THEN 1 ELSE 0 END) AS bd_entre_0_e_120,
        CONCAT(ROUND(((bd_menor_que_0 + bd_maior_que_120) / bd_total) * 100, 1), '%') as taxa_de_fora_da_idade,
        CONCAT(ROUND((bd_entre_0_e_120 / bd_total) * 100, 1), '%') as taxa_de_dentro_da_idade
    FROM tb_members
    group by 1
    order by 1 asc
    """
  ).show()
)

+------+----------------+----------------+--------+--------------+----------------+----------------+---------------------+-----------------------+
| safra|minimo_per_safra|maximo_per_safra|bd_total|bd_menor_que_0|bd_maior_que_120|bd_entre_0_e_120|taxa_de_fora_da_idade|taxa_de_dentro_da_idade|
+------+----------------+----------------+--------+--------------+----------------+----------------+---------------------+-----------------------+
|201601|           -7168|            2016| 4294184|       2397278|             330|         1896576|                55.8%|                  44.2%|
|201602|           -7168|            2016| 4502354|       2575982|             333|         1926039|                57.2%|                  42.8%|
|201603|           -7168|            2016| 4697972|       2745403|             339|         1952230|                58.4%|                  41.6%|
|201604|           -7168|            2016| 4876301|       2901081|             343|         1974877|                59

### OutLiers identificados, Colunas com tipos diferentes identificados sendo necessário transformar para a bronze

Outliers na 'bd'. Possiveis soluções para os outliers: Exclusão, substituição. Analisar a correlação de churn com a idade e ver sua importancia. Para o momento: alterações esperadas para a construção da bronze, agora apenas sinalizar idades invalidas e que precisam ser tratadas.

Entender melhor a coluna de data 'safra', o que ela realmente representa, usar ela ou usar a registred_init_time?

#### Documentar:
Quantos valores foram removidos ou corrigidos?
Que critérios foram usados para identificar os valores inválidos?
Qual o impacto de substituir ou excluir? A quantidade de substituição é grande ou pequena?

In [ ]:
df_year_members = df_members.withColumn('year', year(col('registration_init_time')))
df_year_members.createOrReplaceTempView('tb_members_year')

(spark.sql(
    """
    SELECT
        year,
        min(bd) AS minimo_per_year,
        max(bd) AS maximo_per_year,
        SUM(CASE WHEN bd IS NULL THEN 0 ELSE 1 END) AS bd_total,
        SUM(CASE WHEN bd <= 0 THEN 1 ELSE 0 END) AS bd_menor_que_0,
        SUM(CASE WHEN bd > 120 THEN 1 ELSE 0 END) AS bd_maior_que_120,
        SUM(CASE WHEN bd > 0 AND bd <= 120 THEN 1 ELSE 0 END) AS bd_entre_0_e_120,
        CONCAT(ROUND(((bd_menor_que_0 + bd_maior_que_120) / bd_total) * 100, 1), '%') as taxa_de_fora_da_idade,
        CONCAT(ROUND((bd_entre_0_e_120 / bd_total) * 100, 1), '%') as taxa_de_dentro_da_idade
    FROM tb_members_year
    group by 1
    order by 1 asc
    """
  ).show()
)

+----+---------------+---------------+--------+--------------+----------------+----------------+---------------------+-----------------------+
|year|minimo_per_year|maximo_per_year|bd_total|bd_menor_que_0|bd_maior_que_120|bd_entre_0_e_120|taxa_de_fora_da_idade|taxa_de_dentro_da_idade|
+----+---------------+---------------+--------+--------------+----------------+----------------+---------------------+-----------------------+
|2004|              0|            786|  314808|         77088|              12|          237708|                24.5%|                  75.5%|
|2005|              0|            237|  496188|         75480|              12|          420696|                15.2%|                  84.8%|
|2006|              0|            994|  647436|           732|              12|          646692|                 0.1%|                  99.9%|
|2007|          -3152|           1821| 1077960|           876|              24|         1077060|                 0.1%|                  99.9%|

os outliers, como foi sugerido no enunciado do problema estão não só em 2015, mas concentrados em outros anos tambem.

## 2 - Entendendo a Transaction

### Primeira visão

In [ ]:
spark.sql('select * from tb_transactions LIMIT 10').show()

+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+------+
|                msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel| safra|
+--------------------+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+------+
|+++IZseRRiQS9aaSk...|               38|              410|           1788|              1788|            0|        20151121|              20170104|        0|201511|
|+++snpr7pmobhLKUg...|               41|               30|            149|               149|            1|        20150526|              20150626|        0|201505|
|+++snpr7pmobhLKUg...|               41|               30|            149|               149|            1|        20150926|              20151026|        0|201509|
|++/9R3sX3

Visto valores de plan_list_price iguais ao actual_amount_paid, para valores altos que é incomum. Para valores baixos é entendivel, pode ser o primeiro mes do cliente, mas valores como 1788 é questionavel.


### Entendendo os minimos e máximos das colunas de data

In [ ]:
df_transactions = (df_transactions.withColumn('payment_method_id', col('payment_method_id').cast('int'))
                                  .withColumn('payment_plan_days', col('payment_plan_days').cast('int'))
                                  .withColumn('plan_list_price', col('plan_list_price').cast('float'))
                                  .withColumn('actual_amount_paid', col('actual_amount_paid').cast('float'))
                                  .withColumn('is_auto_renew', col('is_auto_renew').cast('boolean'))
                                  .withColumn('transaction_date', to_date(col('transaction_date'), 'yyyyMMdd'))
                                  .withColumn('membership_expire_date', to_date(col('membership_expire_date'), 'yyyyMMdd'))
                                  .withColumn('is_cancel', col('is_cancel').cast('boolean'))
                                  .withColumn('safra', col('safra').cast('int'))
                                  )
df_transactions.createOrReplaceTempView('tb_transactions')

(spark.sql(
    """
    select min(transaction_date), max(transaction_date),
           min(membership_expire_date), max(membership_expire_date),
           min(safra), max(safra)
    from tb_transactions
    """
  ).show()
)

+---------------------+---------------------+---------------------------+---------------------------+----------+----------+
|min(transaction_date)|max(transaction_date)|min(membership_expire_date)|max(membership_expire_date)|min(safra)|max(safra)|
+---------------------+---------------------+---------------------------+---------------------------+----------+----------+
|           2015-01-01|           2017-02-28|                 1970-01-01|                 2017-03-31|    201501|    201702|
+---------------------+---------------------+---------------------------+---------------------------+----------+----------+



data de expirar o plano em 1970? o primeiro usuario registrado foi em 2004, antes disso pode se considerar uma anomalia.

In [ ]:
(spark.sql(
    """
    select count(*) as quantidade_linhas_antes_de_2004
    from tb_transactions
    where membership_expire_date < '2004-01-01'
    """
  ).show()
)

+-------------------------------+
|quantidade_linhas_antes_de_2004|
+-------------------------------+
|                           1501|
+-------------------------------+



### Entendendo cada coluna da Transaction

In [ ]:
def executa_query_transaction_por_coluna(coluna):
  query = f"""
            select count(distinct {coluna}) as quantidade_distinta_{coluna}
            from tb_transactions
           """
  spark.sql(query).show()

for i in df_transactions.columns:
    print(i)
    executa_query_member_por_coluna(i)

msno
+------------------------+
|quantidade_distinta_msno|
+------------------------+
|                 2363626|
+------------------------+

payment_method_id
+-------------------------------------+
|quantidade_distinta_payment_method_id|
+-------------------------------------+
|                                   40|
+-------------------------------------+

payment_plan_days
+-------------------------------------+
|quantidade_distinta_payment_plan_days|
+-------------------------------------+
|                                   37|
+-------------------------------------+

plan_list_price
+-----------------------------------+
|quantidade_distinta_plan_list_price|
+-----------------------------------+
|                                 51|
+-----------------------------------+

actual_amount_paid
+--------------------------------------+
|quantidade_distinta_actual_amount_paid|
+--------------------------------------+
|                                    57|
+------------------------------

- 40 meios de pagamento
- 37 tamanhos de planos diferentes, (qnt de dias acumulados ativos?)
- 26 safras (201501 a 201702)

### Entendendo os preços dos planos

In [ ]:
(spark.sql(
    """
    select min(plan_list_price), max(plan_list_price),
           min(actual_amount_paid), max(actual_amount_paid),
           SUM(CASE WHEN plan_list_price = actual_amount_paid THEN 1 ELSE 0 END) AS qnt_iguais
    from tb_transactions
    """
  ).show()
)

+--------------------+--------------------+-----------------------+-----------------------+----------+
|min(plan_list_price)|max(plan_list_price)|min(actual_amount_paid)|max(actual_amount_paid)|qnt_iguais|
+--------------------+--------------------+-----------------------+-----------------------+----------+
|                 0.0|              2000.0|                    0.0|                 2000.0|  19114847|
+--------------------+--------------------+-----------------------+-----------------------+----------+



In [ ]:
df_transactions.describe('plan_list_price', 'actual_amount_paid').show()

+-------+------------------+------------------+
|summary|   plan_list_price|actual_amount_paid|
+-------+------------------+------------------+
|  count|          20712225|          20712225|
|   mean|140.24088353617248|142.83468555406287|
| stddev|132.27679209056055|133.60944276809502|
|    min|               0.0|               0.0|
|    max|            2000.0|            2000.0|
+-------+------------------+------------------+



In [ ]:
# vendo quais sao os valores que mais se repetem na coluna plan_list_price e actual_amount_paid
(spark.sql(
    """
      select safra,
             plan_list_price,
             payment_plan_days
      from tb_transactions
      group by 1, 2, 3
      order by 1 desc
      limit 10
    """
  ).show()
)

(spark.sql(
    """
      select safra,
             actual_amount_paid,
             payment_plan_days
      from tb_transactions
      group by 1, 2, 3
      order by 1 desc
      limit 10
    """
  ).show()
)

+------+---------------+-----------------+
| safra|plan_list_price|payment_plan_days|
+------+---------------+-----------------+
|201702|            119|               30|
|201702|              0|              120|
|201702|              0|                3|
|201702|            180|               45|
|201702|             35|                7|
|201702|              0|                7|
|201702|            150|               30|
|201702|              0|              195|
|201702|             99|               30|
|201702|            100|               30|
+------+---------------+-----------------+

+------+------------------+-----------------+
| safra|actual_amount_paid|payment_plan_days|
+------+------------------+-----------------+
|201702|               119|               30|
|201702|                 0|              120|
|201702|                 0|                3|
|201702|               180|               45|
|201702|                35|                7|
|201702|                 0|  

In [ ]:
(spark.sql(
    """
      select safra,
             plan_list_price,
             payment_plan_days,
             count(plan_list_price) as qnt_plan_list_price
      from tb_transactions
      group by 1, 2, 3
      order by 4 desc
      limit 10
    """
  ).show()
)

+------+---------------+-----------------+-------------------+
| safra|plan_list_price|payment_plan_days|qnt_plan_list_price|
+------+---------------+-----------------+-------------------+
|201512|            149|               30|             596373|
|201511|            149|               30|             580714|
|201601|            149|               30|             555415|
|201508|            149|               30|             496022|
|201509|            149|               30|             486624|
|201608|            149|               30|             486141|
|201507|            149|               30|             484580|
|201609|            149|               30|             476899|
|201506|            149|               30|             470793|
|201607|            149|               30|             470265|
+------+---------------+-----------------+-------------------+



possiveis valores dos planos

- 180 -
- 150 -
- 149 -
- 129 -
- 119 -
- 100 -
- 99 -

### Vendo se há valores missing

In [ ]:
(spark.sql(
    """
      SELECT
        SUM(CASE WHEN msno IS NULL THEN 1 ELSE 0 END) AS msno,
        SUM(CASE WHEN payment_method_id IS NULL THEN 1 ELSE 0 END) AS payment_method_id,
        SUM(CASE WHEN payment_plan_days IS NULL THEN 1 ELSE 0 END) AS payment_plan_days,
        SUM(CASE WHEN plan_list_price IS NULL THEN 1 ELSE 0 END) AS plan_list_price,
        SUM(CASE WHEN actual_amount_paid IS NULL THEN 1 ELSE 0 END) AS actual_amount_paid,
        SUM(CASE WHEN is_auto_renew IS NULL THEN 1 ELSE 0 END) AS is_auto_renew,
        SUM(CASE WHEN transaction_date IS NULL THEN 1 ELSE 0 END) AS transaction_date,
        SUM(CASE WHEN membership_expire_date IS NULL THEN 1 ELSE 0 END) AS membership_expire_date,
        SUM(CASE WHEN is_cancel IS NULL THEN 1 ELSE 0 END) AS is_cancel,
        SUM(CASE WHEN safra IS NULL THEN 1 ELSE 0 END) AS safra
      FROM tb_transactions
    """
  ).show()
)

+----+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+-----+
|msno|payment_method_id|payment_plan_days|plan_list_price|actual_amount_paid|is_auto_renew|transaction_date|membership_expire_date|is_cancel|safra|
+----+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+-----+
|   0|                0|                0|              0|                 0|            0|               0|                     0|        0|    0|
+----+-----------------+-----------------+---------------+------------------+-------------+----------------+----------------------+---------+-----+



### Observações e questionamentos

  - Data de vencimento do plano em 1970?
  - Transações de 201501 a 201702
  - 40 meios de pagamentos
  - Maior parte dos registros de preços dos planos, tanto Amount quanto por plano estão com valores iguais.
  - Tipos dos dados estão atrapalhando as consultas em cima dos dados
  - Coluna is_cancel: entender como ela pode ajudar no nosso contexto
  - Não há missing

## 3 - Entendendo a User_logs

### Primeira visão

In [ ]:
df_userlogs.show(5)

+--------------------+------+------+------+------+-------+-------+-------+----------+
|                msno| safra|num_25|num_50|num_75|num_985|num_100|num_unq|total_secs|
+--------------------+------+------+------+------+-------+-------+-------+----------+
|SwlrSivYHoKF9V5wm...|201701| 121.0|  28.0|  14.0|   29.0|  704.0|  827.0|184606.903|
|rE5wSmHEF1Dhu55zh...|201605|  26.0|   2.0|   5.0|    6.0|  462.0|  256.0|119439.485|
|hx+cyaQ/Jcdr/Z5fo...|201611| 161.0|  71.0|  49.0|   34.0|  668.0|  891.0|204791.242|
|53QW6B70J23X2UCvx...|201502|  37.0|   9.0|   3.0|    9.0|  408.0|  447.0|101186.041|
|/0S1N/oRyxGLZlzxn...|201506| 205.0|  49.0|  23.0|   21.0|  225.0|  489.0| 69957.524|
+--------------------+------+------+------+------+-------+-------+-------+----------+
only showing top 5 rows



### Entendendo cada coluna da User_logs

Aqui temos somente variaveis númnericas, nenhuma categorica, nao tendo importância ver a distinção dos valores.

In [ ]:
df_userlogs.describe().show()

+-------+--------------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+--------------------+
|summary|                msno|            safra|           num_25|           num_50|            num_75|           num_985|          num_100|           num_unq|          total_secs|
+-------+--------------------+-----------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+--------------------+
|  count|            26758971|         26758971|         26758971|         26758971|          26758971|          26758971|         26758971|          26758971|            26758971|
|   mean|                null|201572.0403868669|95.42601163549973|24.03566975725636|14.903521327483034|16.538671797207748|450.1598216538297|440.91930526775485|-2.11717488634742...|
| stddev|                null|61.87618074610147|175.2882595645691|39.12554591867812|  22.626721

In [ ]:
df_userlogs.select(min('safra'), max('safra')).show()

+----------+----------+
|min(safra)|max(safra)|
+----------+----------+
|    201501|    201702|
+----------+----------+



### Entendento o total de segundos ouvidos

In [ ]:
df_userlogs_total_secs = df_userlogs.select('total_secs')
df_userlogs_total_secs.show(5)

df_userlogs_total_secs.select(min('total_secs'), max('total_secs')).show()

df_userlogs_total_secs.describe().show()

+----------+
|total_secs|
+----------+
|184606.903|
|119439.485|
|204791.242|
|101186.041|
| 69957.524|
+----------+
only showing top 5 rows

+--------------------+--------------------+
|     min(total_secs)|     max(total_secs)|
+--------------------+--------------------+
|-2.39807672957245...|9.223372037135594E15|
+--------------------+--------------------+

+-------+--------------------+
|summary|          total_secs|
+-------+--------------------+
|  count|            26758971|
|   mean|-2.11717488634742...|
| stddev|6.839043702349416E14|
|    min|-2.39807672957245...|
|    max|9.223372037135594E15|
+-------+--------------------+



#### Analisando em horas e dias ouvidos

In [ ]:
df_userlogs_total_secs_horas = df_userlogs_total_secs.withColumn('horas', col('total_secs') / 3600)
df_userlogs_total_secs_horas.select(min('horas'), max('horas')).show()
df_userlogs_total_secs_dias = df_userlogs_total_secs.withColumn('dias', col('total_secs') / 86400)
df_userlogs_total_secs_dias.select(min('dias'), max('dias')).show()

+--------------------+--------------------+
|          min(horas)|          max(horas)|
+--------------------+--------------------+
|-6.66132424881237...|2.562047788093220...|
+--------------------+--------------------+

+--------------------+--------------------+
|           min(dias)|           max(dias)|
+--------------------+--------------------+
|-2.77555177033848...|1.067519911705508...|
+--------------------+--------------------+



### Vendo se há valores missing

In [ ]:
df_userlogs.select([count(when(col(c).isNull(), c)).alias(c) for c in df_userlogs.columns]).show()

+----+-----+------+------+------+-------+-------+-------+----------+
|msno|safra|num_25|num_50|num_75|num_985|num_100|num_unq|total_secs|
+----+-----+------+------+------+-------+-------+-------+----------+
|   0|    0|     0|     0|     0|      0|      0|      0|         0|
+----+-----+------+------+------+-------+-------+-------+----------+



### Observações e Questionamentos

  - Tipo do dado da data integer, indo de 201501 a 201702
  - Dados como Double
  - Valores negativos para segundos totais de musicas ouvidas? Valores discrepantes para total de segs ouvidos
  - Não há valores faltantes

## 4 - Entendendo o Conjunto. Como são as DataBases e como elas se relacionam?



  1. Members: Registro de todos os usuários do sistema, contém daonde o usuário é, idade, genero, como ele se registrou e qual foi a data que ele entrou no sistema e se ele esta ativo ou não (assinante ou não).

    - **Desafio**: Lidar com a idade dos clientes

  2. Transactions: Transações de todos os clientes quando assinam o plano, contendo data de assinatura, tempo para vencimento, meios de pagamento, valores do plano, total até agora arrecadado de cada cliente. Aqui os clientes se repetiram com uma visão acumulada de casa transação por data.

    - **Desafio**: Lidar com o preço dos planos assinados

  3. User_logs: Logs de todos os usuários, contendo seus comportentos dentro do sistema, com tempo total de músicas ouvidas, e parcelas de músicas ouvidas, podendo ser identificado possiveis musicas preferidas ou não.

    - **Desafio**: Lidar com o total de segundos ouvidos

## Dicionario de dados FINAL

1. Transactions:
  - Colunas:
    - 1º 'mnso': id do usuário (string);
    - 2º 'payment_method_id': Método de pagamento, Númerico (int);
    - 3º 'payment_plan_days': tamanho do plano em DIAS (int);
    - 4º 'plan_list_price': preço do respectivo plano em New Taiwan Dollar (NTD) (Float/Double);
    - 5º 'actual_amount_paid': Total pago atual (nao entendi) (Float/Double);
    - 6º 'is_auto_renew': Renovação automatica (1 ou 0) (int);
    - 7º 'transaction_date': data da transação no formato %Y %M %D (?);
    - 8º 'membership_expire_date': quando o plano vence no formato %Y %M %D;
    - 9º 'is_cancel': se nessa transação o cliente cancelou ou não o plano;
    - 10º 'safra': data (nao entendi 100% ao que se refere essa data).

2. User_logs:
  - Colunas:
    - 1º 'msno': id do usuário;
    - 2º 'safra': formato %Y %M %D (nao entendi essa data);
    - 3º 'num_25': quantidade de músicas tocadas menos de 25% da duração dela;
    - 4º 'num_50': quantidade de músicas tocadas entre 25% e 50% da duração dela;
    - 5º 'num_75': quantidade de músicas tocadas entre 50% e 75% da duração dela;
    - 6º 'num_985': quantidade de músicas tocadas entre 75% e 98,5% da duração dela;
    - 7º 'num_100': quantidade de músicas tocadas acima de 98,5%;
    - 8º 'num_unq': quantidade de musicas tocadas apenas 1 vez;
    - 9º 'total_secs': total de segundos tocados.

3. Members:
  - Colunas:
    - 1º 'msno': id do usuário;
    - 2º 'city': cidade do usuário, Númerico;
    - 3º 'bd': idade, obs: outliers em 2015;
    - 4º 'gender': gênero, Categorico (male or female);
    - 5º 'registered_via': meio de registro do cliente, Númerico;
    - 6º 'registration_init_time': primeiro regristo do cliente formato %Y %M %D, de 2004 a 20161231;
    - 7º 'safra': base do inicio ao fim de 2016;
    - 8º 'is_ativo': se o cliente esta ativo ou não, 0 ou 1;

## Analise de distribuição das variaveis

 - Analise inicial para entender como os valores estão em termo de frequencia, afim de detectar alguma anomalidade, como outliers
 - Volumetria

## Analise de valores missing

## Checagem de integridade da Base

 - Valores duplicados:
 - Missing:
 - Consistencia:

# Construção da Bronze:

- Limpeza e tratamento dos dados

    - Input dos ausentes, se necessário
    - Remoção de outliers, se necessário
    - Normalização e padronização, se necessário
    - Conversão de tipos de dados

- Feature Engineering
    - novas variaveis

## Members Bronze

 - Casting das Variaveis
 - Tratamento das idades
 - Tratamento dos gêneros

In [ ]:
# Tratando os valores da members, convertendo os valores que tem que ser int para int, e etc.

df_members.createOrReplaceTempView('tb_members')

spark.sql('Describe tb_members').show()

spark.sql(
    'Select msno AS id_membro \
             \
    from tb_members'

).show()



+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|                msno|   string|   null|
|               safra|   string|   null|
|registration_init...|   string|   null|
|                city|   string|   null|
|                  bd|   string|   null|
|              gender|   string|   null|
|      registered_via|   string|   null|
|            is_ativo|      int|   null|
+--------------------+---------+-------+



In [ ]:
from pyspark.sql.functions import expr

members_value = members_bronze.select('bd').filter((col('bd') > 0) & (col('bd') <= 120))
members_value.describe().show()

# pegar a mediana do members_value
member_value_mediana = members_value.agg(expr('percentile(bd, 0.5)').alias('mediana')).collect()[0]['mediana']
print(member_value_mediana)

# para valores menores que zero no members_bronze 'bd' atribuir o valor da member_value_mediana
members_bronze = members_bronze.withColumn('bd', when(col('bd') <= 0, member_value_mediana).otherwise(col('bd'))) \
                               .withColumn('bd', when(col('bd') > 120, member_value_mediana).otherwise(col('bd')))

members_bronze.describe().show()

# members_value2 = members_bronze.select('bd').filter((col('bd') <= 0))
# members_value2.describe().show()

# members_value3 = members_bronze.select('bd').filter((col('bd') > 120))
# members_value3.describe().show()

# members_bronze.write.mode("overwrite").parquet(path_bronze_members)


+-------+------------------+
|summary|                bd|
+-------+------------------+
|  count|          24350120|
|   mean|29.883953015426616|
| stddev|11.078740529579315|
|    min|                 1|
|    max|               120|
+-------+------------------+

27.0
+-------+--------------------+------------------+----------------------+-----------------+------------------+--------+------------------+-------------------+
|summary|                msno|             safra|registration_init_time|             city|                bd|  gender|    registered_via|           is_ativo|
+-------+--------------------+------------------+----------------------+-----------------+------------------+--------+------------------+-------------------+
|  count|            63867246|          63867246|              63867246|         63867246|          63867246|25657069|          63867246|           63867246|
|   mean|                null|201606.90368996028|  2.0140061722392727E7|4.369941346774214|28.09954016

# Construção da Silver:

- Analise de Features e seleção
- Construção da Target

# Construção da Gold

- Analytical Base Table (ABT)
- Features e Target estruturados e prontos para irem para treino, validação e teste dos modelos

# Criação dos modelos

qual metodologia seguir

(seleção de variaveis, modelos (por que dos modelos, por que nao explorar outros))

(interpretabilidade X performance)

(caso haja, como foi feita a escolha de hiperparametros)

(tota decisão ser justificada, comparada)

((avaliar performance dos modelos), (estabilidade, performance, explicabilidade), (treinamento de modelos por janelas de tempo, out of time))

(Shaply Values, )